In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1.3)


from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import scipy
from sklearn.linear_model import Ridge

In [ ]:
df = pd.read_csv("/kaggle/input/ecommerce-data/data.csv", encoding="ISO-8859-1", dtype={'CustomerID': str}, parse_dates=['InvoiceDate'])
df.shape

In [ ]:
df.head()

### **EDA**

In [ ]:
missing_percentage =  df.isnull().mean() * 100
missing_percentage

In [ ]:
# Removing None Customer IDs ... save clean dataframe with same variable `df`
df = df.dropna(subset=['CustomerID'])

In [ ]:
# Just for Checking 
df.isna().sum()

### **Q1: Which Country makes the most transactions?**

In [ ]:
sns.set_style('ticks')
plt.figure(figsize=(18,5));
#data =df.Country.value_counts()[:20]
data = df['Country'].value_counts()[:20].sort_values(ascending=False)
pal = sns.light_palette("seagreen",n_colors=len(data))
sns.barplot(x=data.index, y=data, palette=np.array(pal[::-1]))

plt.title('Which Country makes the most transactions?')
plt.yscale('log')
plt.xticks(rotation=90)
plt.show()
#location = df['Country'].value_counts()[:20]
#loation.plot(kind='bar', color="green", title= 'Which Country makes the most transactions?', figsize=(20,5))

In [ ]:
df['Country'].value_counts()[:10].sort_values(ascending=False)

### **Q2: Show the Top 10 Customers how made the largest Revenue**

### **Q6: Calculate the Total Revenue for each Row**

In [ ]:
df['Revenue'] = df['Quantity']*df['UnitPrice']
df_top_10 = {'CustomerID':df.groupby('CustomerID').Revenue.sum().sort_values(ascending=False)[:10].index,
            'Revenue':df.groupby('CustomerID').Revenue.sum().sort_values(ascending=False)[:10].values}
df_top_10 = pd.DataFrame(df_top_10)
df_top_10

### **Q3: Show Top 5 frequent Customers**

In [ ]:
df['Frequency'] = df['CustomerID'].value_counts()[:5]
#df_top_cust = df['CustomerID'].value_counts()[:5]
#df_top_cust= {'CustomerID':df.groupby('CustomerID').Frequency.sum().sort_values(ascending=False)[:5].index,
 #            'Frequency':df.groupby('CustomerID').Frequency.sum().sort_values(ascending=False)[:5].values}
#df_top_cust = pd.DataFrame(df_top_cust)
df_top_cust = df.groupby(['CustomerID']).agg(
    {"Frequency": len}).sort_values("Frequency", ascending=False).head(5).reset_index()
df_top_cust

In [ ]:
f"{df.shape[0]:,}"

### **Q4: Create a histogram that reveals the count of words of 𝑫𝒆𝒔𝒄𝒓𝒊𝒑𝒕𝒊𝒐𝒏**

In [ ]:
#xvals = np.arange(len(df['Description'].value_counts()))
#yvals = list(df['Description'].value_counts())

#counts, bins, _ = plt.hist(df['Description'])

#for n, b in zip(counts, bins):
 #       plt.gca().text(b + 0.1, n, str(n))  # +0.1 to center text

sns.set_style('darkgrid')
plt.figure(figsize=(25,6))
data =df.Description.apply(len).value_counts()

sns.barplot(x=data.index, y=data ,color="purple")
plt.xlabel('DescriptionLength')
plt.ylabel('count')     

#plt.xticks(rotation=90)
plt.show()

###  **Q5: Extract {𝑌𝑒𝑎𝑟, 𝑀𝑜𝑛𝑡ℎ, 𝑊𝑒𝑒𝑘, 𝑊𝑒𝑒𝑘𝐷𝑎𝑦, 𝐷𝑎𝑦, 𝐷𝑎𝑦𝑂𝑓𝑌𝑒𝑎𝑟,𝐷𝑎𝑡𝑒} features from 𝐼𝑛𝑣𝑜𝑖𝑐𝑒𝐷𝑎𝑡𝑒 feature**

In [ ]:
df.drop('Frequency', axis=1, inplace=True)
df.head(2)

In [ ]:
import datetime

df.InvoiceDate = pd.to_datetime(df['InvoiceDate'])
df["Year"] =df.InvoiceDate.dt.year 
df["Quarter"] =df.InvoiceDate.dt.quarter 
df["Month"] =df.InvoiceDate.dt.month 
df["Week"] = df.InvoiceDate.dt.week
df["Weekday"] = df.InvoiceDate.dt.dayofweek
df["Day"] = df.InvoiceDate.dt.day
df["Dayofyear"] = df.InvoiceDate.dt.dayofyear
df["Date"] = df.InvoiceDate.dt.date

In [ ]:
df.head()

### **Q7 & Q8: Aggregate Revenue & Quantity across the grouped_features**

In [ ]:
grouped_features = ['Date','Year','Quarter','Month','Week','Weekday','Dayofyear','Day','StockCode']

In [ ]:
df.groupby(grouped_features,as_index=False)[['Quantity','Revenue']].sum().head()